In [1]:
import numpy as np
import pandas as pd
import datetime
import seaborn as sns
from matplotlib import lines
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import warnings
warnings.filterwarnings('ignore')

In [33]:
listNames = ['Khulan','Adrian','Mathilde','Virgil','FannyM','Margaux','JL','Hélène','Luca','Brice','ThiMai','Ani','Romain','StephB']
_type=['Grasshopper', 'Hybrid', 'Plugin']
niv=['No experience','Novice','Limited','Basic','Advanced','Expert']

In [3]:
def context_reorder(data, choice):
    context_grasshopper = data[(data["Object"]>1.1) & (data["Object"]<2)]
    context_hybrid = data[(data["Object"]>2.1) & (data["Object"]<3)]
    context_plugin = data[(data["Object"]>3.1) & (data["Object"]<4)]
    order1=context_grasshopper.index[0]
    order2=context_hybrid.index[0]
    order3=context_plugin.index[0]
    _order=[_x for _, _x in sorted(zip([order1, order2, order3], [1, 2, 3]))]
    if choice==0:
        return [context_grasshopper,context_hybrid,context_plugin], ['Grasshopper','Hybrid','Plugin'], _order
    elif choice==1:
        return context_grasshopper,'Grasshopper'
    elif choice==2:
        return context_hybrid,'Hybrid'
    elif choice==3:
        return context_plugin,'Plugin'

def sub_selection(_context, choice):
    facade_choice=choice
    if facade_choice==0:
        return _context
    return _context.xs(choice,level='Object',drop_level=False)

def diff_analysis(_context):
    _diff=(_context.diff(1, 0) != 0)
    _diff.iloc[0]=False
    _diff['Time']=_context['Time']
    return _diff

def timeconvert(time):
    hours=int(time / 3600)
    minutes=int((time - hours * 3600) / 60)
    seconds=int(time - hours * 3600 - minutes * 60)
    return datetime.time(hours,minutes,seconds)

def time_to_sec(time):
    return (time.second + time.minute * 60 + time.hour * 3600) / 60

def split_profile(_df, _imp):
    prof1=pd.DataFrame()
    prof2=pd.DataFrame()
    prof3=pd.DataFrame()
    _profile = profile.loc[_df.index]
    score=_profile[_imp]
    it=0
    for x in score:     
        if int(x)==0:
            prof1=prof1.append(_df.iloc[it])
        if int(x)==1:
            prof2=prof2.append(_df.iloc[it])
        if int(x)==2:
            prof3=prof3.append(_df.iloc[it])
        it+=1
    return prof1.T,prof2.T,prof3.T

def plot_feedback(ax, a, _feedback_df):
    if a is None:
        pass
    else:
        _feedback_df=_feedback_df[_feedback_df['Order'] == a]
    colors=_feedback_df['Level'].map(lambda x:_color[int(x)])
    _feedback_df['Level']=_feedback_df['Level'].map(lambda x:niv[int(x)])
    color_set=colors.drop_duplicates().to_list()
    ax.set_autoscaley_on(False)
    ax.set_ybound(0,11)
    r=sns.pointplot(x='variable', y='value', data=_feedback_df[_feedback_df['Survey'] == 'A'], hue='Level', ci='sd', dodge=0.3, palette=color_set, markers='x', ax=ax, linestyles='--')

    _lines=r.get_lines()
    for line in _lines[:]:
        line.set_alpha(0.6)
        line.set_linewidth(1.5)
        line.set_zorder(1)

    sns.pointplot(x='variable', y='value', data=_feedback_df[_feedback_df['Survey'] == 'B'], hue='Level', ci='sd', dodge=0.3, palette=color_set, ax=ax)
    ax.set_ylabel(None)
    ax.set_xlabel(None)
    if a is not None:
        ax.set_xticklabels("")
    ax.get_legend().remove()
    ax.tick_params(axis='y',pad=-1)
    return

def interaction_line_plot(_ax, result, _order, keep_list):
    int1=result[result.index.get_level_values('Order') == _order]
    _min=int1.min()
    _max=int1.max()
    _mean1=int1.mean()
    int1.drop(keep_list, axis=1, inplace=True)
    colors=int1['Level'].map(lambda x:_color[int(x)])
    int1['Level']=int1['Level'].map(lambda x:niv[int(x)])
    color_set=colors.drop_duplicates().to_list()
    int1=int1.melt(id_vars=['Level'])
    sns.pointplot(x=int1['variable'],y=int1['value'],data=int1,hue=int1['Level'],palette=color_set,ax=_ax,ci='sd',dodge=0.2)
    _ax.set_title('Interaction ' + str(_order + 1), loc='left', pad=3)
    _ax.set_ylabel(None)
    _ax.set_xlabel(None)
    _ax.tick_params(axis='y',pad=-1)
    _ax.legend(loc='upper right',title='Level')
    _ax.set_xticklabels("")
    return int1

def box_plot(ax, c, r, keep_list):
    r=r[[keep_list[c], 'Level']]
    r=r.reset_index('Order',drop=False)
    r2=r.copy()
    color_palette=sns.mpl_palette('RdYlBu_r',3)
    color_palette.append('gray')
    r2['Order']=3
    r=r.append(r2)
    sns.boxplot(ax=ax, y='Order', x=keep_list[c], data=r, orient='h', hue='Order', dodge=False, palette=color_palette)
    sns.swarmplot(ax=ax, y='Order', x=keep_list[c], data=r, orient='h', hue='Level', palette='Paired', dodge=True, s=10, marker='x', linewidth=3)
    ax.set_title(keep_list[c], loc='center', pad=3)
    ax.get_legend().remove()
    ax.set_ylabel(None)
    ax.set_yticklabels(['Interaction 1','Interaction 2','Interaction 3','Average'])
    return

def time_level_plot(_df, df_level, order_choice, ax, regression_order):
        if order_choice is None:
            _df=_df.droplevel('Order')
            level_list=df_level
        else:
            _df=_df.xs(order_choice, level='Order')
            level_list=df_level[order_choice]
        for lvl in level_list:
            level_df=_df.xs(lvl,level='Level').melt().applymap(lambda x:int(x))
            sns.regplot(x='variable', y='value', data=level_df, color=_color[lvl], ax=ax, order=regression_order, x_estimator=np.mean, scatter=False, ci=None)
        ax.set_ylabel(None)
        ax.set_xticklabels("")
        ax.set_xlabel(None)
        ax.tick_params(axis='y',pad=-1)
        return


In [4]:
survey=pd.read_excel("Enquete.xlsx")
survey['Nom']=listNames
intro= survey.iloc[:, [4, 7, 10, 16]]
subject_level_df= survey.iloc[:, [4, 21, 24, 27, 30, 33, 36, 39, 42]]
detail= survey.iloc[:, [4, 45, 48]]
interaction1_feedback= survey.iloc[:, [4, 51, 54, 57, 60, 63]]
interaction1_feedback.set_index('Nom', inplace=True)
interaction2_feedback= survey.iloc[:, [4, 69, 72, 75, 78, 81]]
interaction2_feedback.set_index('Nom', inplace=True)
interaction3_feedback= survey.iloc[:, [4, 87, 90, 93, 96, 99]]
interaction3_feedback.set_index('Nom', inplace=True)
posteriori_interaction1_feedback= survey.iloc[:, [4, 105, 108, 111, 114, 117]]
posteriori_interaction1_feedback.set_index('Nom', inplace=True)
posteriori_interaction2_feedback= survey.iloc[:, [4, 123, 126, 129, 132, 135]]
posteriori_interaction2_feedback.set_index('Nom', inplace=True)
posteriori_interaction3_feedback= survey.iloc[:, [4, 141, 144, 147, 150, 153]]
posteriori_interaction3_feedback.set_index('Nom', inplace=True)
col=interaction1_feedback.columns
interaction2_feedback.columns=col
interaction3_feedback.columns=col
posteriori_interaction1_feedback.columns=col
posteriori_interaction2_feedback.columns=col
posteriori_interaction3_feedback.columns=col
interaction_feedback=[interaction1_feedback, interaction2_feedback, interaction3_feedback]
posteriori_interaction_feedback=[posteriori_interaction1_feedback, posteriori_interaction2_feedback, posteriori_interaction3_feedback]
def convert_subject_level(n):
        _level=["Aucune expérience","Novice","Limité","Habitué","Avancé","Expert"]
        return _level.index(n)

subject_level_df.set_index('Nom', inplace=True)
subject_level_df=subject_level_df.applymap(convert_subject_level)
subject_level_df['Tot']=subject_level_df[['Rhino', 'Grasshopper']].mean(axis=1).round(0).convert_dtypes(convert_integer=True)
subject_level_df['Tot'].quantile(q=0.5)
def profile(n):
    low=n.quantile(q=0.33)
    high=n.quantile(q=0.66)
    p=[]
    for x in n:
        if x <= low:
            p.append(0)
        elif low<x<=high:
            p.append(1)
        elif high<x:
            p.append(2)
    return p

profile=subject_level_df.apply(profile, axis=0)

In [41]:
iterations_df=pd.DataFrame()
time_iterations_df=pd.DataFrame()
time_phases_df=pd.DataFrame()
time_unique_df=pd.DataFrame()
phase_df=pd.DataFrame()
unique_df=pd.DataFrame()
feedback_df=pd.DataFrame()
for name in listNames:
    path = 'input/'+ name + '/' + name +'.csv'
    test = pd.read_csv(path,sep=';')
    test["Time"] = pd.to_datetime(test["Time"],format = '%H:%M:%S:%f')
    Implementations,Index,Order = context_reorder(test, 0)
    diff_df_list=[]
    phase=pd.DataFrame()
    int_feedback=[_x for _, _x in sorted(zip(Order, interaction_feedback))]
    int_feedback_post=[_x for _, _x in sorted(zip(Order, posteriori_interaction_feedback))]
    unique=pd.DataFrame()
    _time_iterations=pd.DataFrame()
    _time_phases=pd.DataFrame()
    _time_unique=pd.DataFrame()
    iteration=pd.DataFrame()
    _time=[]
    i=0
    time_intervals=11
    for implementation in Implementations:
        implementation['Object']=implementation['Object'].map(lambda x:((x % 1)*10).__round__(0)).convert_dtypes(convert_integer=True)
        implementation = implementation.set_index(['Object'],append=True)
        implementation = implementation.rename_axis(['int','Object'])
        imp2=sub_selection(implementation,2)
        last2=imp2['Time'].iloc[-1]
        first2=imp2['Time'].iloc[0]
        imp3=sub_selection(implementation,3)
        last3=imp3['Time'].iloc[-1]
        first3=imp3['Time'].iloc[0]
        imp4=sub_selection(implementation,4)
        last4=imp4['Time'].iloc[-1]
        first4=imp4['Time'].iloc[0]
        delta_time2 = last2 - first2
        delta_time3 = last3 - first3
        delta_time4 = last4 - first4
        _time.append(delta_time2.seconds)
        _time.append(delta_time3.seconds)
        _time.append(delta_time4.seconds)
        diff_df_list.append(diff_analysis(sub_selection(implementation,2)).rename_axis([Index[i],'Object']))
        diff_df_list.append(diff_analysis(sub_selection(implementation,3)).rename_axis([Index[i],'Object']))
        diff_df_list.append(diff_analysis(sub_selection(implementation,4)).rename_axis([Index[i],'Object']))
        for k in range(2,5):
            time_unique=pd.DataFrame()
            imp=sub_selection(implementation,k)
            start_time=imp['Time'].iloc[0]
            imp['Time']= imp['Time'] - start_time
            imp['Time']=imp['Time'].map(lambda x:x.seconds)
            start_time=imp['Time'].iloc[0]
            end_time=imp['Time'].iloc[-1]
            timerange=round((end_time / time_intervals), 2)
            for j in range(1, time_intervals):
                _imp2=imp[(imp['Time'] > start_time) & (imp['Time'] < (start_time + timerange * j))]
                _imp2=_imp2.nunique()
                imp1=imp[(imp['Time'] > start_time) & (imp['Time'] < (start_time + timerange * (j + 1)))].nunique()
                imp1=imp1-_imp2
                time_unique=time_unique.append(imp1.drop(['Time']),ignore_index=True)

            time_unique[Index[i]]=time_unique.sum(axis=1).map(lambda _x:int(_x))
            time_unique=time_unique[Index[i]]
            time_unique.rename(k,inplace=True)
            _time_unique=_time_unique.append(time_unique)

        _unique=pd.Series(imp2.drop(['Time'],axis=1).nunique(axis=0),name=Index[i])
        _unique['Object']=2
        _unique['Order']=Order.index(i+1)
        unique=unique.append(_unique)
        _unique=pd.Series(imp3.drop(['Time'],axis=1).nunique(axis=0),name=Index[i])
        _unique['Object']=3
        _unique['Order']=Order.index(i+1)
        unique=unique.append(_unique)
        _unique=pd.Series(imp4.drop(['Time'],axis=1).nunique(axis=0),name=Index[i])
        _unique['Object']=4
        _unique['Order']=Order.index(i+1)
        unique=unique.append(_unique)
        i+=1
    i=0
    
    for diff_df in diff_df_list:
        _type_=diff_df.index.names[0]
        diff_df['Time']-=diff_df['Time'].iloc[0]
        start_time=diff_df['Time'].iloc[0]
        end_time=diff_df['Time'].iloc[-1]
        timerange= (end_time - start_time) / time_intervals
        diff_df.reset_index('Object',inplace=True)
        diff_df.set_index('Time',inplace=True)
        timeit=[]
        if timerange > (start_time-start_time):
            obj=diff_df['Object'][0]
            st=diff_df.resample(timerange, closed='left').sum()
            st[_type_]=st.drop('Object',axis=1).sum(axis=1)
            st=st.reset_index()
            st=st[_type_]
            st.iloc[-2]=st.iloc[-2]+st.iloc[-1]
            st=st.iloc[:-1]
            st['Object']=obj
        else:
            st=pd.DataFrame(data=None,index=range(11))
            st[_type_]=int(diff_df.sum(axis=1).values)
            st=st[_type_]
            display(st)
        _time_iterations=_time_iterations.append(st)
        l=((diff_df * 1).diff(1, 0) == 1)
        if timerange > (start_time-start_time):
            st=l.resample(timerange,closed='left').sum()
            st[_type_]=st.sum(axis=1)
            st=st.reset_index()
            st=st[_type_]
            st.iloc[-2]=st.iloc[-2]+st.iloc[-1]
            st=st.iloc[:-1]
        else:
            st=pd.DataFrame(data=None,index=range(11))
            st[_type_]=int(l.sum(axis=1).values)
            st=st[_type_]
        _time_phases=_time_phases.append(st)
        l=l.sum()
        l['Time']=_time[i]
        l['Object']=obj
        l['Order']=Order.index(_type.index(_type_)+1)
        l.name=_type_
        phase.append(l)
        obj=diff_df['Object'][0]
        t=diff_df.sum()
        t["Total Iterations"]=t.drop(['Object']).sum()
        t['Time']=_time[i]
        t["Iterations / Minute"]=t["Total Iterations"]/(t['Time']/60)
        t['Object']=obj
        t.name=_type_
        t['Order']=Order.index(_type.index(_type_)+1)
        i=i+1
        iteration=iteration.append(t)

    Index=_time_iterations.index
    #display(iteration)
    iterations = pd.DataFrame(iteration,index=Index)
    iterations=iterations.set_index(['Object','Order'],append=True)
    total_iterations = pd.Series(iterations.sum(),name='Tot')
    average_iterations = pd.Series(iterations.mean(),name='Average')
    std_iterations = pd.Series(iterations.std(),name='Standard Deviation')
    iteration=iteration.append(total_iterations)
    iteration=iteration.append(average_iterations)
    iteration=iteration.append(std_iterations)
    phases = pd.DataFrame(phase,index=Index)
    total_phases = pd.Series(phases.sum(),name='Tot')
    avg_phases = pd.Series(phases.mean(),name='Average')
    std_phases = pd.Series(phases.std(),name='Standard Deviation')
    phase=phase.append(total_phases)
    phase=phase.append(avg_phases)
    phase=phase.append(std_phases)
    unique_vals = pd.DataFrame(unique, index=Index)
    tot_unique_vals = pd.Series(unique_vals.sum(),name='Tot')
    avg_unique_vals = pd.Series(unique_vals.mean(),name='Average')
    std_unique_vals = pd.Series(unique_vals.std(),name='Standard Deviation')
    unique.append(tot_unique_vals)
    unique.append(avg_unique_vals)
    unique.append(std_unique_vals)
    display(unique)
    display(phase)
    Name = [name]
    Index2 = pd.MultiIndex.from_product([Name,Index],names=["Name","Type"])
    index_frame2=Index2.to_frame()
    #Index.append("Tot")
    #Index.append("Average")
    #Index.append("Standard Deviation")
    Index = pd.MultiIndex.from_product([Name,Index],names=["Name","Type"])
    index_frame=Index.to_frame()
    prf=""
    for level in subject_level_df.loc[name].values:
        prf=prf+str(int(level)).strip()
    order=[]
    for item in range(1,7):
        if item in Order:
            order.append(int(Order.index(item)))
        else:
            order.append("-")
    index_frame['Order']=order
    index_frame2['Order']= order[:-3]
    index_frame2['Level']=prf
    index_frame2=index_frame2[['Name', 'Level', 'Order', 'Type']]
    Index2=pd.MultiIndex.from_frame(index_frame2)
    index_frame['Level']=prf
    index_frame=index_frame[['Name', 'Level', 'Order', 'Type']]
    Index=pd.MultiIndex.from_frame(index_frame)
    feedback_post=pd.DataFrame([int_feedback_post[0].xs(name), int_feedback_post[1].xs(name), int_feedback_post[2].xs(name)], index=Index2)
    feedback=pd.DataFrame([int_feedback[0].xs(name), int_feedback[1].xs(name), int_feedback[2].xs(name)], index=Index2)
    Index2 = pd.MultiIndex.from_product([Index2,[2,3,4]],names=[['Name', 'Level', 'Order', 'Type'],'Object'])
    _time_iterations=pd.DataFrame(_time_iterations.values, index=Index2)
    _time_phases=pd.DataFrame(_time_phases.values, index=Index2)
    #display(Index2.to_frame())
    #display(_time_unique.rename_axis('Facade'))
    _time_unique=pd.DataFrame(_time_unique.values, index=Index2)

    feedback['Survey']= 'A'
    feedback_post['Survey']= 'B'
    feedback=feedback.append(feedback_post)
    feedback.set_index('Survey', append=True, inplace=True)
    iterations = pd.DataFrame(iteration,index=Index)
    unique = pd.DataFrame(unique,index=Index)
    unique['Max']=unique.max(axis=1)
    unique['Total']=unique.drop('Max',axis=1).sum(axis=1)
    unique['CV']=unique.drop(['Total','Max'],axis=1).std(axis=1)/unique.drop(['Total','Max'],axis=1).mean(axis=1)
    phases = pd.DataFrame(phase,index=Index)
    phases['Max']=phases.drop('Time',axis=1).max(axis=1)
    phases['Total']=phases.drop(['Max','Time'],axis=1).sum(axis=1)
    phases['Average']=phases.drop(['Total','Max','Time'],axis=1).mean(axis=1)

    if iterations_df.shape[0]>1:
        iterations_df = iterations_df.append(iterations)
        phase_df = phase_df.append(phases)
        unique_df = unique_df.append(unique)
        feedback_df = feedback_df.append(feedback)
        time_iterations_df=time_iterations_df.append(_time_iterations)
        time_phases_df=time_phases_df.append(_time_phases)
        time_unique_df=time_unique_df.append(_time_unique)
    elif iterations_df.shape[0]<1 :
        iterations_df=iterations
        phase_df=phases
        feedback_df = feedback
        time_iterations_df = _time_iterations
        time_phases_df = _time_phases
        time_unique_df= _time_unique
iterations_df['Time']=iterations_df['Time'].apply(timeconvert)
phase_df['Time']=phase_df['Time'].apply(timeconvert)
feedback_df.columns=['Accessibility', 'Comprehension', 'Ease of Use', 'Usability', 'Satisfaction']
feedback_df=feedback_df.reset_index(['Type', 'Level', 'Order', 'Survey'], drop=False)
feedback_df=feedback_df.melt(id_vars=['Type', 'Level', 'Order', 'Survey'])

,TileXSize,TileYSize,Variability,Columns,Rows,MultipleX,MultipleY,CladdingSeed,OpeningSeed,MargeX,UpperMargin,LowerMargin,MinOpeningHeight,MinOpeningWidth,DoorNumber,Object,Order
Grasshopper,5.0,5.0,1.0,8.0,6.0,1.0,1.0,17.0,9.0,7.0,8.0,7.0,12.0,6.0,5.0,2.0,0.0
Grasshopper,3.0,1.0,6.0,4.0,5.0,3.0,1.0,10.0,1.0,1.0,13.0,14.0,12.0,1.0,4.0,3.0,0.0
Grasshopper,1.0,1.0,1.0,3.0,5.0,1.0,1.0,8.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,4.0,0.0
Hybrid,1.0,1.0,4.0,1.0,2.0,3.0,2.0,4.0,3.0,1.0,1.0,1.0,4.0,3.0,3.0,2.0,1.0
Hybrid,3.0,1.0,2.0,1.0,1.0,3.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,3.0,3.0,1.0
Hybrid,3.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,6.0,1.0,1.0,2.0,2.0,4.0,1.0
Plugin,1.0,1.0,1.0,1.0,1.0,2.0,1.0,6.0,3.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0
Plugin,1.0,1.0,2.0,1.0,1.0,2.0,1.0,9.0,11.0,1.0,1.0,2.0,2.0,2.0,2.0,3.0,2.0
Plugin,1.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4.0,2.0


""
Tot
Average
Standard Deviation


ValueError: Length of values (6) does not match length of index (9)

In [ ]:
def double_plot(df, clname, ax1, ax2, title, color_list, _level, fn, _color='#bde0fe'):
    
    time=df[clname]
    time=time.reset_index(level=_level)
    time['x']=title
    ax1.margins(y=0.1)
    r=sns.regplot(x=time[_level].map(fn),y=time[clname],ax=ax1,x_estimator=np.mean,order=2)
    _markers=r.get_children()
    _markers[0].set_facecolors(color_list)
    _lines=r.get_lines()
    it=0
    for line in _lines[:-1]:
        line.set_color(color_list[it])
        it+=1
        line.set_zorder(3)
    _markers[0].set_zorder(4)
    ax1.set_xticklabels("")
    ax1.set_ylim(bottom=0)
    ax1.tick_params(axis='x',length=0)
    
    if ax2 is not None:
        sns.violinplot(x='x',y=time[clname],data=time,ax=ax2,positions=[2],inner='quartile',cut=0,color='#EDF6F9',alpha=0.2,zorder=1)
        sns.swarmplot(x='x', y=time[clname], data=time, hue=time[_level], ax=ax2, palette=color_list, edgecolor="grey", linewidth=0.4, dodge=True, s=7)
        ax2.get_legend().remove()
        ax2.set_xticklabels("")
        ax2.set_frame_on(False)
        ax2.grid(visible=True)
        ax2.set_ylabel(None)
        ax2.set_xlabel(None)

    ax1.set_frame_on(False)
    ax1.set_ylabel(None)
    ax1.set_xlabel(None)
    return 1
def choose_level(_df, b):
    _df=_df.reset_index(level='Level')
    _df['Level']=_df['Level'].map(lambda x:int(x[b]))
    level_length=_df['Level'].value_counts().sort_index()
    _df.set_index(['Level'],append=True,inplace=True)
    _df.sort_index(axis=0,level='Level',inplace=True)
    return _df,level_length

In [ ]:
def context_analysis(a, df, time_df, study_columns, b, title, y2=0.175):
    it2=df.xs(_type[a], level='Type')
    if 'Time' in it2.columns:it2['Time']=it2['Time'].apply(time_to_sec)
    it2,level_length = choose_level(it2, b)
    it=it2[study_columns]
    it2=it2.drop(study_columns, axis=1)
    fig=plt.figure(figsize=(22,12),facecolor="#fefdfd")
    _Title=str(_type[a])
    fig.title=str(title)
    gs = GridSpec(6,30,figure=fig)
    ax1=fig.add_subplot(gs[2:4,:3])
    ax3=fig.add_subplot(gs[2:4,4:7])
    ax4=fig.add_subplot(gs[2:4,8:11])
    ax9=fig.add_subplot(gs[4:,12:])
    ax5=fig.add_subplot(gs[4:,12:])
    ax2=fig.add_subplot(gs[2:4,12:],sharex=ax9)
    ax6=fig.add_subplot(gs[4:,:3])
    ax7=fig.add_subplot(gs[4:,4:7])
    ax8=fig.add_subplot(gs[4:,8:11])
    ax10=fig.add_subplot(gs[1:2,:3])
    ax11=fig.add_subplot(gs[1:2,4:7])
    ax12=fig.add_subplot(gs[1:2,8:11])
    ax13=fig.add_subplot(gs[:2,12:30])
    ax14=fig.add_subplot(gs[:1,:12])
    sns.heatmap(it2.droplevel(['Order','Level']),annot=True,ax=ax5,cbar=False,cmap="vlag",linewidth=0.5)
    ax5.set_yticklabels("")
    ax5.set_ylabel(None)
    ax9.set_yticklabels("")
    ax9.set_ylabel(None)
    ax9.set_xticklabels("")
    ax9.set_xlabel(None)
    locs=ax5.get_xticks()
    ax2.set_xticks(locs)
    time=pd.DataFrame(it[study_columns[0]].droplevel(['Level', 'Order']))
    ax9.tick_params(axis='y',tickdir='in')
    tot_it=pd.DataFrame(it[study_columns[1]])
    sns.heatmap(time,ax=ax6,cbar=False,annot=True,cmap="vlag")
    sns.heatmap(tot_it,ax=ax7,cbar=False,annot=True,cmap="vlag",fmt="g")
    ax7.set_yticklabels("")
    ax7.set_ylabel(None)
    ax7.set_xticklabels("")
    ax7.set_xlabel(study_columns[1])
    it_min=pd.DataFrame(it[study_columns[2]])
    sns.heatmap(it_min,ax=ax8,cbar=False,annot=True,cmap="vlag")
    ax8.set_yticklabels("")
    ax8.set_ylabel(None)
    ax8.set_xticklabels("")
    ax8.set_xlabel(study_columns[2])

    level_plot_df=it2.droplevel(['Order'])
    level_plot_df.columns=locs
    colors=sns.mpl_palette("Paired",6)
    level_plot_df=level_plot_df.reset_index(level='Level',drop=False)
    color_set=level_plot_df['Level'].map(lambda _x:colors[_x])
    color_set=color_set.drop_duplicates().to_list()
    levels=['No experience','Novice','Limited','Basic','Advanced','Expert']
    level_plot_df['Level']=level_plot_df['Level'].map(lambda _x:levels[_x])
    ax2lines=ax2.get_lines()
    _it=0
    for line in ax2lines:
        line.set_color(color_set[_it])
        line.set_linewidth(0.5)
        _it+=1
    level_plot_df=level_plot_df.melt(id_vars='Level')
    sns.pointplot(x=level_plot_df['variable'],y=level_plot_df['value'],hue=level_plot_df['Level'],palette=color_set,data=level_plot_df,ax=ax2,dodge=0.5)
    
    context_feedback=feedback_df[feedback_df['Type'] == _type[a - 1]]
    context_feedback['Level']=context_feedback['Level'].map(lambda _x:_x[b])
    plot_feedback(ax14, None, context_feedback)
    ax14.set_frame_on(False)
    
    double_plot(it, study_columns[0], ax10, ax1, 'Time in minutes', color_set, 'Level', (lambda _x:_x))
    double_plot(it, study_columns[1], ax11, ax3, 'Iterations', color_set, 'Level', (lambda _x:_x))
    double_plot(it, study_columns[2], ax12, ax4, 'Iterations/Minute', color_set, 'Level', (lambda _x:_x))
    
    df=time_df.xs(_type[a - 1], level='Type').reset_index('Level', drop=False)
    df['Level']=df['Level'].map(lambda _x:int(_x[b]))
    df_level=df['Level'].unique()
    df.set_index(['Level'],append=True,inplace=True)
    time_level_plot(df, df_level, None, ax13, 3)
    ax13.set_frame_on(False)
    fig.suptitle(title + ": " + _Title, fontsize=20, y=0.92)

    x1=0.125
    x2=0.9
    y1=0.361
    _t=pd.Series([0,0,0,0,0,0])
    level_length=_t.T.add(level_length,fill_value=0.0,axis=0)
    h_cell=(y1-y2)/(len(it[str(study_columns[0])]) - 3)
    _h_cell=h_cell*(level_length.loc[0])
    y2=y1-_h_cell+0.001*2
    x=[(x1,x2),(x2,x1),(x1,x1)]
    y=[(y1,y1),(y2,y2),(y2,y1)]
    line1 = lines.Line2D(x,y,transform=fig.transFigure,c=colors[0],linewidth=2)
    y1=y2-0.001*2
    _h_cell=h_cell*(level_length.loc[1])
    y2=y1-_h_cell+0.001*2
    x=[(x1,x2),(x2,x1),(x1,x1)]
    y=[(y1,y1),(y2,y2),(y2,y1)]
    line2 = lines.Line2D(x,y,transform=fig.transFigure,c=colors[1],linewidth=2)
    y1=y2-0.001*2
    _h_cell=h_cell*(level_length.loc[2])
    y2=y1-_h_cell+0.001*2
    x=[(x1,x2),(x2,x1),(x1,x1)]
    y=[(y1,y1),(y2,y2),(y2,y1)]
    line3 = lines.Line2D(x,y,transform=fig.transFigure,c=colors[2],linewidth=2)
    y1=y2-0.001*2
    _h_cell=h_cell*(level_length.loc[3])
    y2=y1-_h_cell+0.001*2
    x=[(x1,x2),(x2,x1),(x1,x1)]
    y=[(y1,y1),(y2,y2),(y2,y1)]
    line4 = lines.Line2D(x,y,transform=fig.transFigure,c=colors[3],linewidth=2)
    y1=y2-0.001*2
    _h_cell=h_cell*(level_length.loc[4])
    y2=y1-_h_cell+0.001*1
    x=[(x1,x2),(x2,x1),(x1,x1)]
    y=[(y1,y1),(y2,y2),(y2,y1)]
    line5 = lines.Line2D(x,y,transform=fig.transFigure,c=colors[4],linewidth=2)
    y1=y2-0.0015*1
    _h_cell=h_cell*(level_length.loc[5])
    y2=y1-_h_cell+0.0015*0
    x=[(x1,x2),(x2,x1),(x1,x1)]
    y=[(y1,y1),(y2,y2),(y2,y1)]
    line6 = lines.Line2D(x,y,transform=fig.transFigure,c=colors[5],linewidth=2)
    if level_length.loc[5]<0.5:line6.set_linewidth(0)
    fig.lines = line1,line2,line3,line4,line5,line6
    if level_length.loc[0]<0.5:line1.set_linewidth(0)
    fig.lines = line1,line2,line3,line4,line5,line6
    ax2.set_frame_on(False)
    ax2.set_xlabel(None)
    ax2.tick_params(axis='y',tickdir='in')
    ax2.tick_params(axis='x',tickdir='out')
    ax5.tick_params(axis='x',tickdir='out',top=True)
    ax2.set_ylabel(None)
    ax2.set_xticklabels("")
    ax6.set_ylabel(None)
    ax12.set_frame_on(False)
    ax12.set_ylabel(None)
    ax12.set_xlabel(None)
    return 1

# Iterations

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', 12,'display.max_colwidth',50):
    display(iterations_df.round(decimals=2))
iterations_df.round(decimals=2).to_excel('output/Iterations.xlsx')

## Étude Générale

In [ ]:
_color=sns.mpl_palette('Paired',5)
_color2=sns.mpl_palette('RdYlBu_r',3)

In [ ]:
context_analysis(0, iterations_df, time_iterations_df, study_columns=['Time', 'Total Iterations', 'Iterations / Minute'], b=4, title='Number of Iterations')
plt.savefig('output/gh_iterations.png')

In [ ]:
context_analysis(1, iterations_df.drop('Adrian', axis=0, level='Name'), time_iterations_df.drop('Adrian', axis=0, level='Name'), study_columns=['Time', 'Total Iterations', 'Iterations / Minute'], b=8, title='Number of Iterations', y2=0.1785)
plt.savefig('output/hybrid_iterations.png')

In [ ]:
context_analysis(2, iterations_df, time_iterations_df, study_columns=['Time', 'Total Iterations', 'Iterations / Minute'], b=3, title='Number of Iterations')
plt.savefig('output/plugin_iterations.png')

### Total

In [ ]:
def global_analysis(df1, study_columns, _title):
    df=df1.droplevel(['Order'])
    fig =plt.figure(figsize=(24,8),facecolor="#fefdfd")
    fig.suptitle(_title + ': Total', x=0.5, y=0.92, fontsize=20)
    gs = GridSpec(4,40,figure=fig)
    ax1=fig.add_subplot(gs[1:3,:4])
    ax3=fig.add_subplot(gs[1:3,5:9])
    ax4=fig.add_subplot(gs[1:3,10:14])
    ax9=fig.add_subplot(gs[3:,15:])
    ax2=fig.add_subplot(gs[:3,15:],sharex=ax9)
    ax5=fig.add_subplot(gs[3:,15:])
    ax6=fig.add_subplot(gs[3:,:4])
    ax7=fig.add_subplot(gs[3:,5:9])
    ax8=fig.add_subplot(gs[3:,10:14])
    ax10=fig.add_subplot(gs[:1,:4])
    ax11=fig.add_subplot(gs[:1,5:9])
    ax12=fig.add_subplot(gs[:1,10:14])

    a=(df.drop(['Average','Standard Deviation','Tot'],axis=0,level='Type'))

    if 'Time' in a.columns:a['Time']=a['Time'].apply(time_to_sec)

    a=a.droplevel(['Level'])
    df=df.droplevel(['Level'])
    _color3=sns.mpl_palette("RdYlBu",3)
    ghtime=a[study_columns[0]].xs('Grasshopper', level='Type').mean()
    hybridtime=a[study_columns[0]].xs('Hybrid', level='Type').mean()
    plugintime=a[study_columns[0]].xs('Plugin', level='Type').mean()
    av_time=pd.DataFrame([ghtime,hybridtime,plugintime], index=['Grasshopper','Hybrid','Plugin'], columns=[study_columns[0]])
    sns.heatmap(av_time,ax=ax6,cbar=None,annot=True,cmap="vlag")
    ax6.set_yticklabels(['Grasshopper','Hybrid','Plugin'])
    
    double_plot(a, study_columns[0], ax10, ax1, 'Time in minutes', _color3, 'Type', (lambda x:_type.index(x)))
    double_plot(a, study_columns[1], ax11, ax3, 'Tot Iterations', _color3, 'Type', (lambda x:_type.index(x)))
    double_plot(a, study_columns[2], ax12, ax4, 'Iterations/min', _color3, 'Type', (lambda x:_type.index(x)))

    ghtime=a[study_columns[1]].xs('Grasshopper', level='Type').mean()
    hybridtime=a[study_columns[1]].xs('Hybrid', level='Type').mean()
    plugintime=a[study_columns[1]].xs('Plugin', level='Type').mean()
    av_time=pd.DataFrame([ghtime,hybridtime,plugintime], index=['Grasshopper','Hybrid','Plugin'], columns=[study_columns[1]])
    sns.heatmap(av_time,ax=ax7,cbar=None,annot=True,cmap="vlag",fmt='g')
    ax7.set_yticklabels("")
    

    ###It/minute

    ghtime=a[study_columns[2]].xs('Grasshopper', level='Type').mean()
    hybridtime=a[study_columns[2]].xs('Hybrid', level='Type').mean()
    plugintime=a[study_columns[2]].xs('Plugin', level='Type').mean()
    av_time=pd.DataFrame([ghtime,hybridtime,plugintime], index=['Grasshopper','Hybrid','Plugin'], columns=[study_columns[2]])
    sns.heatmap(av_time,ax=ax8,cbar=None,annot=True,cmap="vlag",fmt='g')
    ax8.set_yticklabels("")
    

    ###
    av_gh=df.xs('Grasshopper',level='Type').drop(study_columns, axis=1).mean()
    av_gh.name='GH'
    av_hyb=df.xs('Hybrid',level='Type').drop(study_columns, axis=1).mean()
    av_hyb.name='Hybrid'
    av_plugin=df.xs('Plugin',level='Type').drop(study_columns, axis=1).mean()
    av_plugin.name='Plugin'
    average=pd.DataFrame([av_gh,av_hyb,av_plugin],index=['GH','Hybrid','Plugin'])
    sns.heatmap(average,ax=ax9,cbar=None,annot=True,cmap="vlag",linewidth=0.5)
    sns.heatmap(average,ax=ax5,cbar=None,annot=True,cmap="vlag",linewidth=0.5)
    df=df.loc[:,['Grasshopper','Hybrid','Plugin'],:]
    locs=ax5.get_xticks()
    labels=ax5.get_xticklabels()
    gh=df.drop(study_columns, axis=1)
    # gh=gh.droplevel(['Level'])
    gh.columns=locs
    gh=gh.reset_index(level='Type',drop=False)


    gh=gh.melt(id_vars=['Type'])
    sns.pointplot(x=gh['variable'],y=gh['value'],data=df,hue=gh['Type'],ax=ax2,palette="RdYlBu",dodge=0.2)
    # ax2.set_xticks(locs);
    ax2.set_xlabel(None)
    ax2.legend(loc='upper left')
    ax2.set_xticklabels("")
    ax2.set_frame_on(False)
    ax2.set_ylabel(None)
    # ax2.set_yticklabels("")
    ax5.set_xticklabels(labels)
    ax1.set_frame_on(False)
    ax1.grid(visible=True)
    ax3.set_frame_on(False)
    ax3.grid(visible=True)
    ax4.set_frame_on(False)
    ax4.grid(visible=True)
    ax9.set_yticklabels("")
    ax5.set_yticklabels("")
    return 1

studycolumn=['Time','Total Iterations','Iterations / Minute']
global_analysis(iterations_df, studycolumn, 'Number of Iterations')
plt.savefig('output/global_iterations.png')

## Etude selon l'ordre

### Grasshopper

In [ ]:
def order_analysis(a, df, time_df, keep_list, b, title):
    fig=plt.figure(figsize=(24,12))
    _title=str(_type[a - 1])
    fig.suptitle(title + ": " + _title, y=0.94, fontsize=20)
    gs = GridSpec(4,12,figure=fig)
    ax1=fig.add_subplot(gs[:1,3:6])
    ax2=fig.add_subplot(gs[1:2,3:6])
    ax3=fig.add_subplot(gs[2:3,3:6])
    ax4=fig.add_subplot(gs[:4,:1])
    ax5=fig.add_subplot(gs[3:4,3:6])
    ax6=fig.add_subplot(gs[:4,1:2])
    ax7=fig.add_subplot(gs[:4,2:3])
    ax8=fig.add_subplot(gs[:1,9:])
    ax9=fig.add_subplot(gs[1:2,9:])
    ax10=fig.add_subplot(gs[2:3,9:])
    ax11=fig.add_subplot(gs[3:4,9:])
    ax12=fig.add_subplot(gs[:1,6:9])
    ax13=fig.add_subplot(gs[1:2,6:9])
    ax14=fig.add_subplot(gs[2:3,6:9])
    ax15=fig.add_subplot(gs[3:4,6:9])
    label=(_type[a - 1])
    keep_list=keep_list
    result=df.reset_index('Level',drop=False)
    result['Level']=result['Level'].map(lambda x:x[b])
    result.sort_values('Level',axis=0,inplace=True)
    if 'Time' in result.columns:result['Time']=result['Time'].apply(time_to_sec)
    result=result.xs(label,level='Type')
    context_feedback=feedback_df[feedback_df['Type'] == _type[a - 1]]
    context_feedback['Level']=context_feedback['Level'].map(lambda x:x[b])
    ax=[ax1,ax2,ax3,ax4,ax6,ax7,ax8,ax9,ax10]
    for i in range(3):
        interaction_line_plot(ax[i], result, i, keep_list)
        box_plot(ax[i + 3], i, result, keep_list)
        plot_feedback(ax[i + 6], i, context_feedback)
    ax6.set_yticklabels("")
    ax7.set_yticklabels("")
    df=time_df.xs(_type[a - 1], level='Type').reset_index('Level', drop=False)
    df['Level']=df['Level'].map(lambda x:int(x[b]))
    df0level=df.xs(0,level='Order')['Level'].unique()
    df1level=df.xs(1,level='Order')['Level'].unique()
    df2level=df.xs(2,level='Order')['Level'].unique()
    df_level=[df0level,df1level,df2level]
    df.set_index(['Level'],append=True,inplace=True)
    ax=[ax12,ax13,ax14]
    for i in range(3):
        time_level_plot(df, df_level, i, ax[i], 3)
    df=df.droplevel('Level')
    for i in range(3):
        sns.regplot(x='variable',y='value',data=df.xs(i,level='Order').melt().applymap(lambda x:int(x)),ax=ax15,order=3,x_estimator=np.mean,scatter=False,color=_color2[i],ci=None)
    ax15.tick_params(axis='y',pad=-1)
    ax15.set_ylabel(None)
    ax15.set_xlabel(None)
    level_plot=result.drop(keep_list,axis=1).drop(['Level'],axis=1)
    level_plot=level_plot.reset_index('Order',drop=False)
    level_plot['Order']=level_plot['Order'].map(lambda x:x+1)
    level_plot=level_plot.melt(id_vars='Order')
    sns.pointplot(x='variable',y='value',data=level_plot,hue='Order',palette='RdYlBu_r',ax=ax5,ci='sd',dodge=0.2)
    sns.pointplot(x='variable',y='value',data=context_feedback[context_feedback['Survey']=='B'],hue='Order',ci='sd',dodge=0.15,palette='RdYlBu_r',ax=ax11)
    ax11.set_ylabel(None)
    ax11.get_legend().remove()
    ax11.set_xlabel(None)
    ax11.tick_params(axis='x',rotation=20)
    ax11.tick_params(axis='y',pad=-1)
    ax5.legend(loc='upper right')
    ax5.tick_params(axis='y',pad=-1)
    ax5.set_xticklabels("")
    ax5.set_xlabel('Parameter')
    ax5.set_title('Average by Interaction order',loc='left',y=1,pad=3)
    return

In [ ]:
keeplist1 =['Time','Total Iterations','Iterations / Minute']
order_analysis(1, iterations_df, time_iterations_df, keeplist1, 4, 'Number of Iterations')
plt.savefig('output/gh_order_iterations.png')

### Hybrid

In [ ]:
order_analysis(2, iterations_df.drop('Adrian', axis=0, level='Name'), time_iterations_df.drop('Adrian', axis=0, level='Name'), keeplist1, 8, 'Number of Iterations')
plt.savefig('output/hybrid_order_iterations.png')

### Plugin

In [ ]:
order_analysis(3, iterations_df, time_iterations_df, keeplist1, 3, 'Number of Iterations')
plt.savefig('output/plugin_order_iterations.png')

# Phases

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', 12,'display.max_colwidth',50):
    display(phase_df.round(decimals=2))
iterations_df.round(decimals=2).to_excel('output/testexcel.xlsx')

## Étude Générale

### Grasshopper

In [ ]:
context_analysis(0, phase_df, time_phases_df, study_columns=['Time', 'Total', 'Max'], b=4, title='Number of modification phases')
plt.savefig('output/gh_phases.png')

In [ ]:
context_analysis(1, phase_df.drop('Adrian', axis=0, level='Name'), time_phases_df.drop('Adrian', axis=0, level='Name'), study_columns=['Time', 'Total', 'Max'], b=8, title='Number of modification phases', y2=0.1785)
plt.savefig('output/hybrid_phases.png')

### Plugin

In [ ]:
context_analysis(2, phase_df, time_phases_df, study_columns=['Time', 'Total', 'Max'], b=3, title='Number of modification phases')
plt.savefig('output/plugin_phases.png')

### Total 

In [ ]:
global_analysis(phase_df, ['Time', 'Total', 'Max'], 'Number of modification phases')
plt.savefig('output/global_phases.png')

## Etude selon l'ordre

### Grasshopper

In [ ]:
keeplist2 =['Time','Total','Max']
order_analysis(1, phase_df, time_phases_df, keeplist2, 4, 'Number of modification phases')
plt.savefig('output/gh_order_phases.png')

### Hybrid

In [ ]:
order_analysis(2, phase_df.drop('Adrian', axis=0, level='Name'), time_phases_df.drop('Adrian', axis=0, level='Name'), keeplist2, 8, 'Number of modification phases')
plt.savefig('output/hybrid_order_phases.png')

### Plugin

In [ ]:
order_analysis(3, phase_df, time_phases_df, keeplist2, 3, 'Number of modification phases')
plt.savefig('output/plugin_order_phases.png')

# Unique

In [ ]:
#Iterations.round(decimals=1)
#ax = plt.subplot(111, frame_on=False)
#ax.xaxis.set_visible(False)  # hide the x axis
#ax.yaxis.set_visible(False)
#ax.figure.set_figwidth(22)
#v=table(ax,Iterations,loc='top')
with pd.option_context('display.max_rows', None, 'display.max_columns', 12,'display.max_colwidth',50):
    display(unique_df.round(decimals=2))
unique_df.round(decimals=2).to_excel('output/testexcel-unique.xlsx')

## Étude Générale

### Grasshopper

In [ ]:
context_analysis(0, unique_df, time_unique_df, study_columns=['CV', 'Total', 'Max'], b=4, title='Number of unique values', y2=0.1785)
plt.savefig('output/gh_unique.png')

In [ ]:
context_analysis(1, unique_df.drop('Adrian', axis=0, level='Name'), time_unique_df.drop('Adrian', axis=0, level='Name'), study_columns=['CV', 'Total', 'Max'], b=8, title='Number of unique values', y2=0.1825)
plt.savefig('output/hybrid_unique.png')

### Plugin

In [ ]:
context_analysis(2, unique_df, time_unique_df, study_columns=['CV', 'Total', 'Max'], b=3, title='Number of unique values', y2=0.1785)
plt.savefig('output/plugin_unique.png')

### Total 

In [ ]:
global_analysis(unique_df, ['CV', 'Total', 'Max'], 'Number of unique values')
plt.savefig('output/global_unique.png')

## Etude selon l'ordre

### Grasshopper

In [ ]:
keeplist3 =['CV','Total','Max']
order_analysis(1, unique_df, time_unique_df, keeplist3, 4, 'Number of unique values')
plt.savefig('output/gh_order_unique.png')

### Hybrid

In [ ]:
order_analysis(2, unique_df.drop('Adrian', axis=0, level='Name'), time_unique_df.drop('Adrian', axis=0, level='Name'), keeplist3, 8, 'Number of unique values')
plt.savefig('output/hybrid_order_unique.png')

### Plugin

In [ ]:
order_analysis(3, unique_df, time_unique_df, keeplist3, 3, 'Number of unique values')
plt.savefig('output/plugin_order_unique.png')

In [ ]:
phase_df.round(decimals=2)